In [1]:
# !pacman -S firefox firefox-i18n-r  geckodriver
# !pip install selenium

In [2]:
# import re
from datetime import datetime as dt
from tqdm.notebook import tqdm
import pandas as pd

In [3]:
import sys
import logging

logging.basicConfig(
        format=u'[%(levelname)-8s] %(asctime)s | %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        level=logging.INFO,
        # level=logging.DEBUG,
        stream=sys.stdout,
    )

In [4]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.float_format', '{:.2f}'.format)

In [5]:
profile_path = '/home/mechanoid/.mozilla/firefox/p144xo2m.default-release'

# url = (
# 'https://www.avito.ru/yaroslavskaya_oblast/avtomobili/chevrolet/niva'
# '?bt=0'
# '&i=1'
# '&pmax=150000'
# '&pmin=10000'
# )

url = 'https://www.avito.ru/sevastopol/kvartiry/prodam?'# '&pmax=5000000'

---

In [8]:
# from lib.avito import AvitoDownloader

# df, html = AvitoDownloader(profile_path).load(url,page_limit=3)

----

In [ ]:
%%time 

from lib.avito import AvitoDownloaderRealty

df, html = AvitoDownloaderRealty(profile_path).load(url)
df.sample(5)

In [ ]:
# for i,h in enumerate(html): 
#     with open(f'tmp/{i:02d}.html','wt') as f: 
#         f.write(h)

In [ ]:
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df.to_excel(f'data/avito_{ts}_raw.xlsx',index=False)

ts

----

In [ ]:
from lib.avito import AvitoDataCleanerRealty

In [ ]:
df_ = AvitoDataCleanerRealty.transform(df)
df_

In [ ]:
ts = dt.now().strftime('%Y-%m-%d_%H-%M')
df_.to_excel(f'data/avito_{ts}.xlsx',index=False)

ts

-----